    This script takes the notebook with RNA and DNA BSID's and collects information for the corresponding samples from fusion summary files, breakpoint density files, GISTIC CNA broad_values file and FPKM files

In [2]:
import argparse
import pandas as pd
import numpy as np 
import zipfile
import statistics
import  scipy
from scipy import stats

In [11]:
# Reading all the input files 
zip=zipfile.ZipFile("/Users/kogantit/Documents/OpenPBTA/OpenPBTA-analysis/data/pbta-cnv-cnvkit-gistic.zip")
CNA=pd.read_csv(zip.open("2019-12-10-gistic-results-cnvkit/broad_values_by_arm.txt"), sep="\t")
CNA = CNA.set_index("Chromosome Arm")

gsva = pd.read_csv("/Users/kogantit/Documents/OpenPBTA/OpenPBTA-analysis/analyses/gene-set-enrichment-analysis/results/gsva_scores_stranded.tsv", sep="\t")
gsva_NFKB = gsva.loc[gsva['hallmark_name'] == "HALLMARK_TNFA_SIGNALING_VIA_NFKB"]
gsva_NFKB = gsva_NFKB.set_index("Kids_First_Biospecimen_ID")

fpkm_df = pd.read_csv("/Users/kogantit/Documents/OpenPBTA/OpenPBTA-analysis/analyses/molecular-subtyping-EPN/epn-subset/epn-pbta-gene-expression-rsem-fpkm-collapsed.stranded.tsv.gz", sep = "\t")
fpkm_df = fpkm_df.set_index("GENE")
zscore_fpkm_df = fpkm_df.apply(scipy.stats.zscore)

fusion = pd.read_csv("/Users/kogantit/Documents/OpenPBTA/OpenPBTA-analysis/analyses/fusion-summary/results/fusion_summary_ependymoma_foi.tsv", sep="\t")
fusion = fusion.set_index("Kids_First_Biospecimen_ID")

breakpoint_density = pd.read_csv("/Users/kogantit/Documents/OpenPBTA/OpenPBTA-analysis/analyses/chromosomal-instability/breakpoint-data/union_of_breaks_densities.tsv", sep="\t")
breakpoint_density = breakpoint_density.set_index("samples")

EPN_notebook = pd.read_csv("/Users/kogantit/Documents/OpenPBTA/OpenPBTA-analysis/analyses/molecular-subtyping-EPN/results/EPN_molecular_subtype.tsv", sep="\t")


In [23]:
# This function takes in a GISTIC  broad_values 
# and a string (loss/gain) and returns 0/1 accordingly 
def DNA_samples_fill_df(CNA_value, loss_gain):
          if CNA_value<0 and loss_gain=="loss":
            return(1)
          elif loss_gain=="gain" and CNA_value>0:
            return(1)
          else:
            return(0)

In [24]:
# Function to generate Z-scores column for every gene 
def fill_df_with_fpkm_zscores(df,fpkmdf, column_name, gene_name):
        zscore_list = scipy.stats.zscore(np.array(df.apply(lambda x: fpkmdf.loc[gene_name, x["Kids_First_Biospecimen_ID_RNA"]], axis=1)))
        df[column_name] = pd.Series(zscore_list)
        return(df)

In [25]:
# Input notebook before adding columns 
EPN_notebook.head()

,Kids_First_Participant_ID,sample_id,Kids_First_Biospecimen_ID_DNA,Kids_First_Biospecimen_ID_RNA,disease_group
0,PT_S4H6KA09,7316-2134,BS_K6A9Z04J,BS_07ANYSYQ,supratentorial
1,PT_164RNWTT,7316-1078,BS_5D24XV4T,BS_0BXY0F9N,infratentorial
2,PT_V3Q78E6F,7316-455,BS_7RQCH5Y7,BS_0QYS36NR,None
3,PT_Y6Y9JJ9P,7316-425,NaN,BS_0WQJP6ZG,supratentorial
4,PT_82A9SDRN,7316-2313,BS_NWYBD9CA,BS_0XEG6SNV,supratentorial


In [26]:
# Input. CNA file 
CNA.head()

,BS_XEVMEYFS,BS_9R82A3VT,BS_6XHDR1WS,BS_7M7JNG00,BS_41SWNWAG,BS_2W200YK5,BS_X1R0N8QS,BS_E7HEQZ1K,BS_CP435MAC,BS_H8MAKDRB,...,BS_3VKW5988,BS_CBMAWSAR,BS_DVDT4VXQ,BS_M5FM63EB,BS_9P4NDTKJ,BS_Y74XAFJX,BS_AHAXPFG3,BS_ZSH09N84,BS_NNFDFAFM,BS_AF5D41PD
Chromosome Arm,,,,,,,,,,,,,,,,,,,,,
1p,2.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1q,2.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,2.0,2.0,1.0,0.0,1.0,0.0,0.0
2p,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0
2q,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0
3p,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
#Adding columns  to EPN_notebook based on values from CNA file (boolean value)
# Matching based on DNA BSID (row names in CNA file and column names in EPN_notebook) -> Look at row 4 below

EPN_notebook["1q_loss"] = EPN_notebook.apply(lambda x: DNA_samples_fill_df(CNA.loc["1q", x["Kids_First_Biospecimen_ID_DNA"]], "loss") 
    if x["Kids_First_Biospecimen_ID_DNA"] is not np.nan else 0,axis=1)

EPN_notebook.head()

,Kids_First_Participant_ID,sample_id,Kids_First_Biospecimen_ID_DNA,Kids_First_Biospecimen_ID_RNA,disease_group,1q_loss
0,PT_S4H6KA09,7316-2134,BS_K6A9Z04J,BS_07ANYSYQ,supratentorial,0
1,PT_164RNWTT,7316-1078,BS_5D24XV4T,BS_0BXY0F9N,infratentorial,0
2,PT_V3Q78E6F,7316-455,BS_7RQCH5Y7,BS_0QYS36NR,None,0
3,PT_Y6Y9JJ9P,7316-425,NaN,BS_0WQJP6ZG,supratentorial,0
4,PT_82A9SDRN,7316-2313,BS_NWYBD9CA,BS_0XEG6SNV,supratentorial,0


In [44]:
#. Similar to the above, adding more columns to EPN_notebook
EPN_notebook["9p_loss"] = EPN_notebook.apply(lambda x: DNA_samples_fill_df(CNA.loc["9p", x["Kids_First_Biospecimen_ID_DNA"]], "loss") 
    if x["Kids_First_Biospecimen_ID_DNA"] is not np.nan else 0,axis=1)
EPN_notebook["9q_loss"] = EPN_notebook.apply(lambda x: DNA_samples_fill_df(CNA.loc["9q", x["Kids_First_Biospecimen_ID_DNA"]], "loss") 
    if x["Kids_First_Biospecimen_ID_DNA"] is not np.nan else 0,axis=1)
EPN_notebook["6p_loss"] = EPN_notebook.apply(lambda x: DNA_samples_fill_df(CNA.loc["6p", x["Kids_First_Biospecimen_ID_DNA"]], "loss") 
    if x["Kids_First_Biospecimen_ID_DNA"] is not np.nan else 0,axis=1)
EPN_notebook["6q_loss"] = EPN_notebook.apply(lambda x: DNA_samples_fill_df(CNA.loc["6q", x["Kids_First_Biospecimen_ID_DNA"]], "loss") 
    if x["Kids_First_Biospecimen_ID_DNA"] is not np.nan else 0,axis=1)
EPN_notebook["11q_loss"] = EPN_notebook.apply(lambda x: DNA_samples_fill_df(CNA.loc["11q", x["Kids_First_Biospecimen_ID_DNA"]], "loss") 
    if x["Kids_First_Biospecimen_ID_DNA"] is not np.nan else 0,axis=1)
EPN_notebook["11q_gain"] = EPN_notebook.apply(lambda x: DNA_samples_fill_df(CNA.loc["11q", x["Kids_First_Biospecimen_ID_DNA"]], "gain") 
    if x["Kids_First_Biospecimen_ID_DNA"] is not np.nan else 0,axis=1)

EPN_notebook.head(4)


,Kids_First_Participant_ID,sample_id,Kids_First_Biospecimen_ID_DNA,Kids_First_Biospecimen_ID_RNA,disease_group,1q_loss,9p_loss,9q_loss,6p_loss,6q_loss,...,NFKB_pathway_GSEAscore,breaks_density-chromosomal_instability,RELA_expr_Z-scores,L1CAM_expr_Zscore,ARL4D_expr_Zscore,CLDN1_expr_zscore,CXorf67_expr_zscore,TKTL1_expr_zscore,GPBP1_expr_zscore,IFT46_expr_zscore
0,PT_S4H6KA09,7316-2134,BS_K6A9Z04J,BS_07ANYSYQ,supratentorial,0,0,0,0,0,...,-0.172490,0.860995,0.684676,0.502920,0.173122,-0.216872,-0.723258,-0.380332,0.228280,-0.791927
1,PT_164RNWTT,7316-1078,BS_5D24XV4T,BS_0BXY0F9N,infratentorial,0,0,0,0,0,...,-0.497694,0.55241,-0.661110,-0.528678,0.957077,-0.254374,0.625133,-0.183613,-1.125898,0.007798
2,PT_V3Q78E6F,7316-455,BS_7RQCH5Y7,BS_0QYS36NR,None,0,0,0,0,0,...,0.704506,0.857433,-0.431582,-0.540579,0.152461,-0.260496,0.265562,0.461415,-1.589245,-0.845350
3,PT_Y6Y9JJ9P,7316-425,NaN,BS_0WQJP6ZG,supratentorial,0,0,0,0,0,...,0.472755,NA,-0.133856,-0.293984,0.906574,-0.223760,-0.749355,-0.380332,0.616315,-1.020594


In [39]:
gsva_NFKB.head(3)
# GSVA. score for NFKB  score 

,hallmark_name,gsea_score
Kids_First_Biospecimen_ID,,
BS_014EVM2D,HALLMARK_TNFA_SIGNALING_VIA_NFKB,0.492365
BS_02NZT8CE,HALLMARK_TNFA_SIGNALING_VIA_NFKB,0.376014
BS_03FT4S8B,HALLMARK_TNFA_SIGNALING_VIA_NFKB,0.463960


In [38]:
# Adding column for NFKB GSEA_score to EPN_notebook
# If DNA sample BSID not found, then fill with "NA" 
EPN_notebook["breaks_density-chromosomal_instability"] = EPN_notebook.apply(lambda x: breakpoint_density.loc[x["Kids_First_Biospecimen_ID_DNA"], "breaks_density"] 
    if x["Kids_First_Biospecimen_ID_DNA"] is not np.nan  else "NA", axis=1)

EPN_notebook.head(3)

,Kids_First_Participant_ID,sample_id,Kids_First_Biospecimen_ID_DNA,Kids_First_Biospecimen_ID_RNA,disease_group,1q_loss,9p_loss,9q_loss,6p_loss,6q_loss,11q_loss,11q_gain,NFKB_pathway_GSEAscore,breaks_density-chromosomal_instability
0,PT_S4H6KA09,7316-2134,BS_K6A9Z04J,BS_07ANYSYQ,supratentorial,0,0,0,0,0,1,0,-0.172490,0.860995
1,PT_164RNWTT,7316-1078,BS_5D24XV4T,BS_0BXY0F9N,infratentorial,0,0,0,0,0,0,0,-0.497694,0.55241
2,PT_V3Q78E6F,7316-455,BS_7RQCH5Y7,BS_0QYS36NR,None,0,0,0,0,0,0,0,0.704506,0.857433


In [37]:
# Printing. FPKM dataframe 
fpkm_df.head(2)

,BS_07ANYSYQ,BS_0BXY0F9N,BS_0QYS36NR,BS_0WQJP6ZG,BS_0XEG6SNV,BS_196BBNEJ,BS_1J2WQ08M,BS_34201DKZ,BS_3F4FQJMR,BS_3X9WMZGJ,...,BS_W80TJC27,BS_WFD0GE5X,BS_WGK8ZMVA,BS_WRHDP7WF,BS_WYERXS4G,BS_X67GTEM6,BS_XF0ZX2B3,BS_XQYHPBFS,BS_YE1MAQYJ,BS_Z7890YNR
GENE,,,,,,,,,,,,,,,,,,,,,
RN7SL1,498860.73,1166393.35,924130.91,491887.43,221443.92,558311.22,794911.03,587404.68,339377.49,909279.15,...,489408.99,566865.79,545791.66,538223.96,233198.63,843876.20,581925.69,490805.68,773334.92,287026.67
RN7SL2,331260.17,629512.02,478657.34,434076.13,150132.00,358347.93,428758.57,474597.97,527067.86,493319.94,...,544324.93,418041.11,354182.15,333834.83,409093.19,546592.37,588427.71,632913.55,519803.67,264058.01


In [45]:
# Adding FPKM for different genes to EPN_notebook using function fill_df_with_fpkm_zscores

EPN_notebook = fill_df_with_fpkm_zscores(EPN_notebook, fpkm_df, "RELA_expr_Z-scores", "RELA")
EPN_notebook = fill_df_with_fpkm_zscores(EPN_notebook, fpkm_df, "L1CAM_expr_Zscore", "L1CAM")
EPN_notebook = fill_df_with_fpkm_zscores(EPN_notebook, fpkm_df, "ARL4D_expr_Zscore", "ARL4D")
EPN_notebook = fill_df_with_fpkm_zscores(EPN_notebook, fpkm_df, "CLDN1_expr_zscore", "CLDN1")
EPN_notebook = fill_df_with_fpkm_zscores(EPN_notebook, fpkm_df, "CXorf67_expr_zscore", "CXorf67")
EPN_notebook = fill_df_with_fpkm_zscores(EPN_notebook, fpkm_df, "TKTL1_expr_zscore", "TKTL1")
EPN_notebook = fill_df_with_fpkm_zscores(EPN_notebook, fpkm_df, "GPBP1_expr_zscore", "GPBP1")
EPN_notebook = fill_df_with_fpkm_zscores(EPN_notebook, fpkm_df,  "IFT46_expr_zscore", "IFT46")


EPN_notebook.head(4)


# Finally print out the dataframe to an output file 


,Kids_First_Participant_ID,sample_id,Kids_First_Biospecimen_ID_DNA,Kids_First_Biospecimen_ID_RNA,disease_group,1q_loss,9p_loss,9q_loss,6p_loss,6q_loss,...,NFKB_pathway_GSEAscore,breaks_density-chromosomal_instability,RELA_expr_Z-scores,L1CAM_expr_Zscore,ARL4D_expr_Zscore,CLDN1_expr_zscore,CXorf67_expr_zscore,TKTL1_expr_zscore,GPBP1_expr_zscore,IFT46_expr_zscore
0,PT_S4H6KA09,7316-2134,BS_K6A9Z04J,BS_07ANYSYQ,supratentorial,0,0,0,0,0,...,-0.172490,0.860995,0.684676,0.502920,0.173122,-0.216872,-0.723258,-0.380332,0.228280,-0.791927
1,PT_164RNWTT,7316-1078,BS_5D24XV4T,BS_0BXY0F9N,infratentorial,0,0,0,0,0,...,-0.497694,0.55241,-0.661110,-0.528678,0.957077,-0.254374,0.625133,-0.183613,-1.125898,0.007798
2,PT_V3Q78E6F,7316-455,BS_7RQCH5Y7,BS_0QYS36NR,None,0,0,0,0,0,...,0.704506,0.857433,-0.431582,-0.540579,0.152461,-0.260496,0.265562,0.461415,-1.589245,-0.845350
3,PT_Y6Y9JJ9P,7316-425,NaN,BS_0WQJP6ZG,supratentorial,0,0,0,0,0,...,0.472755,NA,-0.133856,-0.293984,0.906574,-0.223760,-0.749355,-0.380332,0.616315,-1.020594
